In [11]:
# import os
# os.chdir('..')

In [12]:
import requests
import pandas as pd
import time
from credentials import CLIENT_ID, CLIENT_SECRET # TEMPORARY WORKAROUND
from constants import DATA_DIR

from access_tokens import get_access_token
from query_spotify import get_tracks_audio_features
from utils import chunk_list

In [13]:
CHUNK_SIZE = 2 # tracks per query
QUERY_DELAY = 60*3 # 3 mins in seconds

In [14]:
## Get track ids
tracks = pd.read_csv(f'{DATA_DIR}/dummy_songs.csv')
track_chunks = chunk_list(tracks.track_id.values, CHUNK_SIZE)
tracks.head(2)

,track_id,track_name,artist_id,artist_name,release_date,release_date_precision,album_len,duration_ms,explicit,track_popularity
0,1x1XQqhBViz4opcpwc7FVs,Reelin' In The Years,6P7H3ai06vU1sGvdpBwDmE,Steely Dan,1972-11-01,day,10,277467,False,75
1,3IvTwPCCjfZczCN2k4qPiH,Dirty Work,6P7H3ai06vU1sGvdpBwDmE,Steely Dan,1972-11-01,day,10,190065,False,73


In [17]:
## Loop through each chunk of artists
for i,ids in enumerate(track_chunks):
    ## Make sure access token is fresh
    token = get_access_token(CLIENT_ID, CLIENT_SECRET, timeout = 60, retries = 3)
    
    ## Get artist data from ids
    print('Getting tracks')
    audio_res = get_tracks_audio_features(ids, access_token = token)
    if(audio_res is None):
        print(f'Error getting batch')
        continue
    
    ## Clean response into dictionary
    audio_res_clean = {
        'track_id': [audio['id'] for audio in audio_res['audio_features']],
        'danceability': [audio['danceability'] for audio in audio_res['audio_features']],
        'energy': [audio['energy'] for audio in audio_res['audio_features']],
        'key': [audio['key'] for audio in audio_res['audio_features']],
        'loudness': [audio['loudness'] for audio in audio_res['audio_features']],
        'mode': [audio['mode'] for audio in audio_res['audio_features']],
        'speechiness': [audio['speechiness'] for audio in audio_res['audio_features']],
        'acousticness': [audio['acousticness'] for audio in audio_res['audio_features']],
        'instrumentalness': [audio['instrumentalness'] for audio in audio_res['audio_features']],
        'liveness': [audio['liveness'] for audio in audio_res['audio_features']],
        'valence': [audio['valence'] for audio in audio_res['audio_features']],
        'tempo': [audio['tempo'] for audio in audio_res['audio_features']],
        'time_signature': [audio['time_signature'] for audio in audio_res['audio_features']],
    }

    ## Convert to dataframe and write to CSV
    print('Writing data to CSV...')
    audio_data = pd.DataFrame.from_dict(audio_res_clean)
    audio_data.columns = audio_res_clean.keys()

    display(audio_data)

    break
    audio_data.to_csv(f'{DATA_DIR}/audio_feature_data.csv', mode='a', index=False)

    print(f'Finished chunk {i+1} of {len(artists.artist_id.values) // CHUNK_SIZE}')
    time.sleep(QUERY_DELAY)

print('Script complete')

Using existing token...
Getting tracks
https://api.spotify.com/v1/audio-features?ids=0xLrMI4R2xTwRHWV3UQCAZ,1tjK9HhmpEfk0ijyIKsoUk
Getting audio feature data...
API request succeeded!
Writing data to CSV...


,track_id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,0xLrMI4R2xTwRHWV3UQCAZ,0.641,0.408,2,-14.971,1,0.0409,0.8240,0.002200,0.0958,0.580,116.436,4
1,1tjK9HhmpEfk0ijyIKsoUk,0.952,0.301,2,-13.936,1,0.0612,0.0126,0.000931,0.0387,0.728,119.629,4


Script complete
